In [60]:

%pprint

Pretty printing has been turned OFF


In [4]:

html_path = r'../data/html/aoestats.io_stats.html'

In [23]:

import pandas as pd

win_rate_df = pd.read_html(html_path)[0]
win_rate_df.set_index('Civilization', drop=True, inplace=True)
for column_name in ['Win Rate', 'Play Rate']:
    win_rate_df[column_name] = win_rate_df[column_name].map(lambda x: float(x.split('%')[0]))

def get_hours(x):
    
    return int(x.split(':')[0])

def get_minutes(x):
    
    return int(x.split(':')[1])

def get_seconds(x):
    
    return int(x.split(':')[0])
    
for column_name in ['Feudal Time', 'Castle Time', 'Imperial Time', 'Game Length']:
    win_rate_df['{} Hours'.format(column_name)] = win_rate_df[column_name].map(get_hours)
    win_rate_df['{} Minutes'.format(column_name)] = win_rate_df[column_name].map(get_minutes)
    win_rate_df['{} Seconds'.format(column_name)] = win_rate_df[column_name].map(get_seconds)
win_rate_df.sample(10).T

Civilization,Koreans,Aztecs,Persians,Chinese,Franks,Spanish,Huns,Saracens,Indians,Magyars
Win Rate,45.31,54.05,50.62,46.98,60.61,51.15,56.47,41.21,53.46,50.9
Play Rate,7.03,7.05,6.69,6.82,7.13,6.98,7.55,6.81,5.7,5.88
Score,5897,5872,6675,6421,6678,6362,6294,6160,6531,6229
Research,18,20,21,22,21,20,21,20,22,22
Feudal Time,0:11:05,0:11:47,0:10:54,0:11:29,0:11:00,0:11:12,0:10:59,0:11:22,0:10:28,0:10:50
Castle Time,0:25:23,0:23:39,0:23:05,0:23:57,0:23:05,0:23:31,0:23:39,0:23:54,0:22:58,0:23:45
Imperial Time,0:41:16,0:39:09,0:38:23,0:39:35,0:38:06,0:39:53,0:40:11,0:40:01,0:38:12,0:40:44
Game Length,0:41:15,0:39:46,0:41:55,0:41:20,0:41:32,0:41:48,0:40:44,0:41:21,0:41:10,0:40:29
Feudal Time Hours,0,0,0,0,0,0,0,0,0,0
Feudal Time Minutes,11,11,10,11,11,11,10,11,10,10


In [24]:

win_rate_df.columns

Index(['Win Rate', 'Play Rate', 'Score', 'Research', 'Feudal Time',
       'Castle Time', 'Imperial Time', 'Game Length', 'Feudal Time Hours',
       'Feudal Time Minutes', 'Feudal Time Seconds', 'Castle Time Hours',
       'Castle Time Minutes', 'Castle Time Seconds', 'Imperial Time Hours',
       'Imperial Time Minutes', 'Imperial Time Seconds', 'Game Length Hours',
       'Game Length Minutes', 'Game Length Seconds'],
      dtype='object')

In [32]:

from sklearn.ensemble import RandomForestRegressor

columns_list = ['Play Rate', 'Score', 'Research', 'Feudal Time Hours',
       'Feudal Time Minutes', 'Feudal Time Seconds', 'Castle Time Hours',
       'Castle Time Minutes', 'Castle Time Seconds', 'Imperial Time Hours',
       'Imperial Time Minutes', 'Imperial Time Seconds', 'Game Length Hours',
       'Game Length Minutes', 'Game Length Seconds']
X = win_rate_df[columns_list].values
y = win_rate_df['Win Rate'].values
regr = RandomForestRegressor(max_depth=2, n_estimators=100)
regr.fit(X, y)
sorted([(cn, ci) for cn, ci in zip(columns_list, regr.feature_importances_)], key=lambda x: x[1],
       reverse=True)

[('Play Rate', 0.5585070054116297), ('Score', 0.22863451842832322), ('Research', 0.14295139126459666), ('Imperial Time Minutes', 0.0336146008289729), ('Game Length Minutes', 0.02335270324786462), ('Castle Time Minutes', 0.01029075379282806), ('Feudal Time Minutes', 0.0026490270257847515), ('Feudal Time Hours', 0.0), ('Feudal Time Seconds', 0.0), ('Castle Time Hours', 0.0), ('Castle Time Seconds', 0.0), ('Imperial Time Hours', 0.0), ('Imperial Time Seconds', 0.0), ('Game Length Hours', 0.0), ('Game Length Seconds', 0.0)]

In [33]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 RandomForestRegressor	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 X	 attempt_to_pickle	 
column_name	 columns_list	 example_iterrows	 get_column_descriptions	 get_hours	 get_minutes	 get_seconds	 html_path	 load_csv	 
load_dataframes	 load_object	 math	 os	 pd	 pickle	 regr	 save_dataframes	 store_objects	 
sys	 win_rate_df	 y	 


In [61]:

selector_str = '#Britons > div.Overview > ul > li.bonuses > ul'
xpath_str = '//*[@id="Britons"]/div[1]/ul/li[5]/ul'
html_str = '''<li>
    Civilization bonuses:
    <ul>
        <li>Town Centers cost -50% wood starting in the Castle Age.</li>
        <li>Foot archers (except Skirmishers) have +1/+2 range in the Castle/Imperial Age.</li>
        <li>Shepherds work 25% faster.</li>
    </ul>
</li>'''
html_folder = os.path.join(DATA_FOLDER, 'html')
file_path = os.path.join(html_folder, 'aoe2_civ_bonuses.html')
with open(file_path, 'rb') as f:
    html_doc = f.read().decode('utf-8')

In [62]:

from bs4 import BeautifulSoup

soup = BeautifulSoup(html_doc, 'html.parser')

In [68]:

for civ in soup.find_all('h3')[:3]:
    print(civ, civ.next_element)

<h3><span>Britons</span></h3> <span>Britons</span>
<h3><span>Byzantines</span></h3> <span>Byzantines</span>
<h3><span>Celts</span></h3> <span>Celts</span>


In [67]:

soup.select(selector_str)

[<ul class="bonuses">
<li>Town Centers cost -50% wood starting in the Castle Age.</li>
<li>Foot archers (except Skirmishers) have +1/+2 range in the Castle/Imperial Age.</li>
<li>Shepherds work 25% faster.</li>
</ul>]

In [ ]:

sub_selector = 'div.Overview > ul > li.bonuses > ul'
civ_name_list = []
for civ_div in soup.find_all('div', id=True):
    civ_name = civ_div.get_attribute_list('id')[0]
    print(civ_name)
    bonuses_ul = civ_div.select(sub_selector)[0]
    for bonus_li in bonuses_ul.find_all('li'):
        if bonus_li.string:
            print(bonus_li.string)
    print()

In [ ]:

sub_selector = 'div.Overview > ul > li.bonuses > ul'
for civ_div in soup.find_all('div', id=True):
    civ_name = civ_div.get_attribute_list('id')[0]
    bonuses_ul = civ_div.select(sub_selector)[0]
    for bonus_li in bonuses_ul.find_all('li'):
        if bonus_li.string:
            print(bonus_li.string)

In [85]:

sub_selector = 'div.Overview > ul > li.bonuses > ul'
civ_name_list = []
for civ_div in soup.find_all('div', id=True):
    civ_name = civ_div.get_attribute_list('id')[0]
    civ_name_list.append(civ_name.strip())

In [99]:

csv_folder = os.path.join(DATA_FOLDER, 'csv')
file_path = os.path.join(csv_folder, 'aoe2_civ_features.csv')
features_df.to_csv(file_path)

In [166]:

import urllib.request

civ_soup_dict = {}
for civ_name in civ_name_list:
    try:
        civ_url = 'https://ageofempires.fandom.com/wiki/{}_(Age_of_Empires_II)'.format(civ_name)
        with urllib.request.urlopen(civ_url) as response:
            civ_html = response.read()
    except:
        civ_url = 'https://ageofempires.fandom.com/wiki/{}'.format(civ_name)
        with urllib.request.urlopen(civ_url) as response:
            civ_html = response.read()
    civ_soup = BeautifulSoup(civ_html, 'html.parser')
    civ_soup_dict[civ_name] = civ_soup

In [ ]:

for i, civ_name in enumerate(civ_soup_dict):
    print(civ_name)
    bonuses_selector = '#Civilization_bonuses'
    bonuses_span = civ_soup_dict[civ_name].select(bonuses_selector)[0]
    bonuses_ul = list(bonuses_span.next_elements)[6]
    for bonus_li in bonuses_ul.find_all('li'):
        print(bonus_li.text.strip())
    print()

In [173]:

txt_folder = os.path.join(DATA_FOLDER, 'txt')
file_path = os.path.join(txt_folder, 'aoe2_civ_features.txt')
with open(file_path, 'wb') as f:
    f.write('\n'.encode('utf-8'))
    for i, civ_name in enumerate(civ_soup_dict):
        bonuses_selector = '#Civilization_bonuses'
        bonuses_span = civ_soup_dict[civ_name].select(bonuses_selector)[0]
        bonuses_ul = list(bonuses_span.next_elements)[6]
        for bonus_li in bonuses_ul.find_all('li'):
            f.write('{}\n'.format(bonus_li.text.strip()).encode('utf-8'))


Make sure to clean up the list before running the cell below to load it back in.

In [174]:

import re

line_regex = re.compile('[\r\n]+', re.MULTILINE)
with open(file_path, 'rb') as f:
    feature_list = line_regex.split(f.read().decode('utf-8').strip())

In [175]:

space_regex = re.compile('\s+')
feature_list = [space_regex.sub('_', feature.strip()) for feature in feature_list]

In [176]:

features_df = pd.DataFrame(data=None, index=civ_name_list, columns=feature_list)

In [177]:

xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'aoe2_civ_features.xlsx')
features_df.to_excel(file_path)